In [108]:
using NCDatasets
using DIVAnd
using PyPlot
using Dates
using Glob
using Test
const plt = PyPlot
include("./config.jl");
include("./ME4OH.jl")
doplot = false

false

In [132]:
timeperiod = timeperiod2

2005:2014

In [123]:
function get_year(datafile::AbstractString)
    NCDataset(datafile, "r") do ds
        theyear = Int64(ds["en4_ymd"][1,1])
        return theyear
    end
end

get_year (generic function with 1 method)

In [137]:
yearlist = zeros(Int64, length(datafilelist))
for (iii, datafile) in enumerate(datafilelist)
    yearlist[iii] = get_year(datafile)
end
yearlist

282-element Vector{Int64}:
 1979
 1979
 1979
 1979
 1979
 1979
 1979
 1979
 1979
 1979
 1979
 1979
 1980
    ⋮
 2001
 2001
 2001
 2001
 2001
 2001
 2002
 2002
 2002
 2002
 2002
 2002

In [136]:
goodyear = findall((yearlist .<= timeperiod2[end]) .& (yearlist .>= timeperiod2[1]))

Int64[]

## Create list of netCDF files
The files with the "nonan" extension were moved to another directory.

In [139]:
datafilelist = Glob.glob("*.nc", datadir)
@info("Found $(length(datafilelist)) files");
datafilelist

[ Info: Found 282 files


282-element Vector{String}:
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197901.update.nc"
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197902.update.nc"
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197903.update.nc"
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197904.update.nc"
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197905.update.nc"
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197906.update.nc"
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197907.update.nc"
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197908.update.nc"
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197909.update.nc"
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197910.update.nc"
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197911.update.nc"
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197912

## Experiment B
We perform purely 2D interpolations by looping on the monthly periods and the depth levels.     
The land-sea mask is created on the 3 specified depth levels.

In [9]:
depth_dohc = [depthlayer1[end], depthlayer2[end], depthlayer3[end]]

3-element Vector{Float64}:
  286.6
  685.9
 1985.3

In [10]:
xi, yi, mask_dohc = DIVAnd.load_mask(bathyfile, true, longrid, latgrid, depth_dohc);        
_, (pm, pn), (xi, yi) = DIVAnd_rectdom(longrid, latgrid);

### Prepare the time grid 
According to the time period specified in `config.jl`.

In [22]:
timegrid1 = get_timegrid(timeperiod1);

## Perform analysis
### Create the netCDF files that will store the results
The output directory is specified in `config.jl`.

In [114]:
outputfilelist = []
for depthlayer in [depthlayer1, depthlayer2, depthlayer3]
    fname = make_fname(timeperiod1, depthlayer, "A")
    outputfile = joinpath(outputdir, fname)
    isfile(outputfile) ? rm(outputfile) : @debug("OK")
    create_netcdf_results(outputfile, longrid, latgrid, timegrid1)
    
    # Add the residual variables in the netCDF
    add_residuals(outputfile, datafilelist)
    
    # Add file to the list
    push!(outputfilelist, outputfile)
end

### Loop on the files and the depths

In [115]:
for (ntime, datafile) in enumerate(datafilelist)
    
    # Read observations
    lon, lat, dates, depth, T, S, dohc = read_profile(datafile)
    
    # Modify coordinates
    lon[lon .< 20.] .+= 360
    
    # Loop on the 3 depth levels
    for ii = 1:1
        
        # Select good values (no NaNs)
        obsval = dohc[ii,:]
        goodval = findall(.!isnan.(obsval));
        ngood = length(goodval)
        
        # Perform interpolation
        fi, s = DIVAndrun(mask_dohc[:,:,ii], (pm, pn), (xi, yi), (Float64.(lon[goodval]), 
        Float64.(lat[goodval])), Float64.(obsval[goodval] .- mean(obsval[goodval])), 
        (5., 5.), 10.; moddim=[1,0])
        
        # Compute residuals
        dataresidual = DIVAnd_residual(s, fi)
        
        # Write field and residual in the netCDF
        NCDataset(outputfilelist[ii], "a") do ds
            ds["dohc"][:,:,ntime] = fi
            ds["obslon"][1:ngood,ntime] = lon[goodval]
            ds["obslat"][1:ngood,ntime] = lat[goodval]
            ds["obstime"][1:ngood,ntime] = dates[goodval]
            ds["dohc_residuals"][1:ngood,ntime] = dataresidual
        end
        
        # Make plot
        if doplot
            fig = plt.figure(figsize=(15, 8))
            ax = plt.subplot(121)
            ax.scatter(lon[goodval], lat[goodval], s=3, c=obsval[goodval])
            ax.set_xlim(20., 380.)
            ax = plt.subplot(122)
            ax.pcolormesh(longrid, latgrid, fi' .+ mean(obsval[goodval]))
            ax.set_xlim(20., 380.)
            plt.show()
        end
    end
end